In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
#print(train_data["Age"].isnull().sum())

#mod_train_data = train_data.dropna(subset=['Age'])
mod_train_data = train_data
tr_med = mod_train_data['Age'].median(skipna=True);
#print("median ",tr_med)
mod_train_data.fillna(tr_med,inplace=True)
mod_train_data['Sex'].replace(['male','female'],[1,2],inplace=True)

mod_test_data = test_data
ts_med = mod_test_data['Age'].median(skipna=True);
#print("median ",ts_med)
mod_test_data.fillna(ts_med,inplace=True)
mod_test_data['Sex'].replace(['male','female'],[1,2],inplace=True)

mod_train_data.to_csv('mod_train_data.csv',index=False)
mod_test_data.to_csv('mod_test_data.csv',index=False)

In [2]:
X_train, y_train = mod_train_data.drop(columns = 'Survived'), mod_train_data['Survived']
X_val, y_val = mod_test_data.drop(columns = 'Survived'), mod_test_data['Survived']


In [3]:
class Node:
    def __init__(self):
        
        # links to the left and right child nodes
        self.right = None
        self.left = None
        
        # derived from splitting criteria
        self.column = None
        self.threshold = None
        
        # probability for object inside the Node to belong for each of the given classes
        self.probas = None
        # depth of the given node
        self.depth = None
        
        # if it is the root Node or not
        self.is_terminal = False

In [4]:
class DecisionTreeClassifier:
    def __init__(self, max_depth = 3, min_samples_leaf = 1, min_samples_split = 2):
        
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.min_samples_split = min_samples_split
        
        self.classes = None
        
        # Decision tree itself
        self.Tree = None
    
    def nodeProbas(self, y):
        '''
        Calculates probability of class in a given node
        '''
        
        probas = []
        
        # for each unique label calculate the probability for it
        for one_class in self.classes:
            proba = y[y == one_class].shape[0] / y.shape[0]
            probas.append(proba)
        return np.asarray(probas)

    def gini(self, probas):
        '''
        Calculates gini criterion
        '''
        
        return 1 - np.sum(probas**2)
    
    def calcImpurity(self, y):
        return self.gini(self.nodeProbas(y))
    
    def calcBestSplit(self, X, y):
        '''
        Calculates the best possible split for the internal node of the tree
        '''
        
        bestSplitCol = None
        bestThresh = None
        bestInfoGain = -999
        
        impurityBefore = self.calcImpurity(y)
        
        # for each column in X
        for col in range(X.shape[1]):
            x_col = X[:, col]
            
            # for each value in the column
            for x_i in x_col:
                threshold = x_i
                y_right = y[x_col > threshold]
                y_left = y[x_col <= threshold]
                
                if y_right.shape[0] == 0 or y_left.shape[0] == 0:
                    continue
                    
                # calculate impurity for the right and left nodes
                impurityRight = self.calcImpurity(y_right)
                impurityLeft = self.calcImpurity(y_left)
                
                # calculate information gain
                infoGain = impurityBefore
                infoGain -= (impurityLeft * y_left.shape[0] / y.shape[0]) + (impurityRight * y_right.shape[0] / y.shape[0])
                
                # is this infoGain better then all other?
                if infoGain > bestInfoGain:
                    bestSplitCol = col
                    bestThresh = threshold
                    bestInfoGain = infoGain
                    
        
        # if we still didn't find the split
        if bestInfoGain == -999:
            return None, None, None, None, None, None
        
        # making the best split
        
        x_col = X[:, bestSplitCol]
        x_left, x_right = X[x_col <= bestThresh, :], X[x_col > bestThresh, :]
        y_left, y_right = y[x_col <= bestThresh], y[x_col > bestThresh]
        
        return bestSplitCol, bestThresh, x_left, y_left, x_right, y_right
                
                
                
    
    def buildDT(self, X, y, node):
        '''
        Recursively builds decision tree from the top to bottom
        '''
        
        # checking for the terminal conditions
        
        if node.depth >= self.max_depth:
            node.is_terminal = True
            return
        
        if X.shape[0] < self.min_samples_split:
            node.is_terminal = True
            return
        
        if np.unique(y).shape[0] == 1:
            node.is_terminal = True
            return
        
        # calculating current split
        splitCol, thresh, x_left, y_left, x_right, y_right = self.calcBestSplit(X, y)
        
        #Leaf node has no splitting feature
        if splitCol is None:    
            node.is_terminal = True

        #Leaf nodes can also have too less samples    
        if x_left.shape[0] < self.min_samples_leaf or x_right.shape[0] < self.min_samples_leaf: 
            node.is_terminal = True
            return
        
        node.column = splitCol
        node.threshold = thresh
        
        # creating left and right child nodes
        node.left = Node()
        node.left.depth = node.depth + 1
        node.left.probas = self.nodeProbas(y_left)
        
        node.right = Node()
        node.right.depth = node.depth + 1
        node.right.probas = self.nodeProbas(y_right)
        
        # splitting recursively
        self.buildDT(x_right, y_right, node.right)
        self.buildDT(x_left, y_left, node.left)
        
        
        
        
    
    def fit(self, X, y):
        '''
        Fitting the training data to model
        '''
        
        if type(X) == pd.DataFrame:
            X = np.asarray(X)
        
        self.classes = np.unique(y)
        # root node creation
        self.Tree = Node()
        self.Tree.depth = 1
        self.Tree.probas = self.nodeProbas(y)
        
        self.buildDT(X, y, self.Tree)
    
    def predictSample(self, x, node):
        '''
        Passes one object through decision tree and return the probability of it to belong to each class
        '''
       
    
        # if we have reached the terminal node of the tree
        if node.is_terminal:
            return node.probas
        
        if x[node.column] > node.threshold:
            probas = self.predictSample(x, node.right)
        else:
            probas = self.predictSample(x, node.left)
            
        return probas
        
        
    
    def predict(self, X):
        '''
        Returns the labels for each X
        '''
        
        if type(X) == pd.DataFrame:
            X = np.asarray(X)
            
        predictions = []
        for x in X:
            pred = np.argmax(self.predictSample(x, self.Tree))
            predictions.append(pred)
        
        return np.asarray(predictions)

In [5]:
model = DecisionTreeClassifier(max_depth = 10, min_samples_leaf=2, min_samples_split=2)
model.fit(X_train, y_train)


CPU times: user 28.4 s, sys: 88.3 ms, total: 28.5 s
Wall time: 28.5 s


In [6]:
def F1(y_pred, y_val):
  tp = fp = tn = fn = 0
  for i in range(len(y_val)):
    if (y_pred[i]==1 and y_val[i]==1):
      tp+=1
    elif (y_pred[i]==1 and y_val[i]==0):
      fp+=1
    elif (y_pred[i]==0 and y_val[i]==1):
      fn+=1
    else:
      tn +=1
  prec = float(tp)/float(tp+fp)   #precision
  rec = float(tp)/float(tp+fn)    #recall
  f1 = 2*float(prec)*float(rec)/float(prec+rec)
  return f1


y_pred = model.predict(X_val)
test_score = float(sum(y_pred == y_val))/ float(len(y_val))
test_loss = int(sum(y_pred != y_val))
train_score = float(sum(model.predict(X_train) == y_train))/ float(len(y_train))
train_loss = int(sum(model.predict(X_train) != y_train))
print("Training Accuracy is ", train_score)
print("Test Accuracy is ", test_score)
print("Training Loss is ", train_loss)
print("Testing Loss is ", test_loss)

train_f1 = F1(model.predict(X_train),y_train)
print("Training F1 score is ", train_f1)
test_f1 = F1(y_pred,y_val)
print("Testing F1 score is ", test_f1)



Training Accuracy is  0.8870967741935484
Test Accuracy is  0.8265682656826568
Training Loss is  70
Testing Loss is  47
Training F1 score is  0.8444444444444443
Testing F1 score is  0.7539267015706805
